# T2D benchmark

In [8]:
!cat ../../resources/config-dbpedia.toml

datadir = "data"
resourcedir = "resources"
cache = false
#executor = "dask"

[kbs]
    [kbs.dbpedia_t2ksubset]
    class = "RDFSearcher"
    labelProperties = ["http://dbpedia.org/ontology/title"]
    store = {class = "Trident", configuration = 'kb/dbpedia_t2ksubset/db/'}

    [kbs.dbpedia_2014_part2]
    class = "RDFSearcher"
    labelProperties = ["http://dbpedia.org/ontology/title"]
    store = {class = "Trident", configuration = 'kb/dbpedia/2014_part2/'}

    [kbs.dbpedia_2016]
    class = "RDFSearcher"
    labelProperties = ["http://dbpedia.org/ontology/title"]
    store = {class = "Trident", configuration = 'kb/dbpedia/2016-10/'}
    
    [kbs.dbpedialookup]
    class = "DBpediaLookup"
    

[assets]
    [assets.t2d-v1]
    class = "T2D"
    download = [
        "http://webdatacommons.org/webtables/tables_instance.tar.gz",
        "http://webdatacommons.org/webtables/entities_instance.tar.gz",
        "http://webdatacommons.org/webtables/classes_instance.csv",
        "http://web

In [9]:
!cat ../../resources/pipelines/t2d-v2-baseline.toml

name = 't2d-v2-baseline'
datadir = 'data'
executor = 'tqdm'

[[step]]
step = "dataset"
params = "t2d-v2"
#sample = 10

[[step]]
step = "link"
usecols = "keycol"
lookup_config = {}
    [step.linker_config]
    class = "Salient"
    searcher = {name = "dbpedia_t2ksubset"}
    #only_majority = "@type"
    

[[step]]
step = "score"
labels = "t2d-v2"
report = ["pdf", "md"]

In [17]:
!cd ../../ && time takco run -v -C resources/config-dbpedia.toml resources/pipelines/t2d-v2-baseline.toml

INFO:root:Set log level to 20
INFO:root:Loaded config {'name': 'config-dbpedia', 'datadir': 'data', 'resourcedir': 'resources', 'cache': False, 'kbs': {'dbpedia_t2ksubset': {'class': 'RDFSearcher', 'labelProperties': ['http://dbpedia.org/ontology/title'], 'store': {'class': 'Trident', 'configuration': 'kb/dbpedia_t2ksubset/db/'}}, 'dbpedia_2014_part2': {'class': 'RDFSearcher', 'labelProperties': ['http://dbpedia.org/ontology/title'], 'store': {'class': 'Trident', 'configuration': 'kb/dbpedia/2014_part2/'}}, 'dbpedia_2016': {'class': 'RDFSearcher', 'labelProperties': ['http://dbpedia.org/ontology/title'], 'store': {'class': 'Trident', 'configuration': 'kb/dbpedia/2016-10/'}}, 'dbpedialookup': {'class': 'DBpediaLookup'}}, 'assets': {'t2d-v1': {'class': 'T2D', 'download': ['http://webdatacommons.org/webtables/tables_instance.tar.gz', 'http://webdatacommons.org/webtables/entities_instance.tar.gz', 'http://webdatacommons.org/webtables/classes_instance.csv', 'http://webdatacommons.org/webtab